In [1]:
import torch
import cv2
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
%run helpers.ipynb
%run NN.ipynb
np.set_printoptions(precision=2)
torch.manual_seed(42)
np.random.seed(42)
#test

E:\Anaconda\envs\triangle_env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


# Hyper parameters

In [3]:
canvas_width = 51
canvas_height= 51
center = np.array([(canvas_height-1) / 2, (canvas_width-1) / 2])
num_drawings = 1000
num_samples = 200
bins = np.arange(0,21)

ts = np.array([0.17,0.2,0.08,0.1])

# Generate the dataset

In [4]:
canvass_train = np.zeros([num_drawings,canvas_height, canvas_width])
canvass_test = np.zeros([num_drawings,canvas_height, canvas_width])

In [5]:
for i in range(num_drawings):
    if np.random.rand() > 0.5:
        canvass_train[i] = draw_polygon(canvass_train[2],3,5)
    else:
        canvass_train[i] = draw_ellipse(canvass_train[i],10,15)

for i in range(num_drawings):
    if np.random.rand() > 0.5:
        canvass_test[i] = draw_polygon(canvass_test[2],3,5)
    else:
        canvass_test[i] = draw_ellipse(canvass_test[i],10,15)

# Get the length vectors

In [6]:
lengthss_train = get_length_encoding(canvass_train,num_samples,bins)
lengthss_test = get_length_encoding(canvass_test,num_samples,bins)

# Turn image into graph

In [7]:
lap_mats_train = adj_mat_to_lap_mat(imgs_to_adj_mat(canvass_train))
lap_mats_test = adj_mat_to_lap_mat(imgs_to_adj_mat(canvass_test))

<class 'networkx.utils.decorators.argmap'> compilation 8:4: FutureWarning: laplacian_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
<class 'networkx.utils.decorators.argmap'> compilation 8:4: FutureWarning: laplacian_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.


# Compute eigenvalues & eigenvectors &hks

In [8]:
evals_train,Us_train = lap_mat_to_eigen(lap_mats_train)
hkss_train = batch_heat_kernel_signature(ts, evals_train, Us_train)

evals_test,Us_test = lap_mat_to_eigen(lap_mats_test)
hkss_test = batch_heat_kernel_signature(ts, evals_test, Us_test)

# Set up Neural Network

In [9]:
class HeatKernelSignatureNet(nn.Module):
    #takes in image (51 x 51), returns heat kernel signature at [0.17,0.47,0.58,0.81]

    def __init__(self,image_width,image_height):
        super().__init__()
        self.image_width = image_width
        self.image_height = image_height

        self.pool = nn.MaxPool2d(2, 2)

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=3, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3)
        self.conv3 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=3)

        self.MLP = MLP(48,4,10,100)


    def forward(self, x):
        x = self.pool(torch.tanh(self.conv1(x)))
        x = self.pool(torch.tanh(self.conv2(x)))
        x = self.pool(torch.tanh(self.conv3(x)))

        x = torch.flatten(x,start_dim=1,end_dim=-1)
        x = self.MLP(x)

        return x

In [10]:
hksNet = HeatKernelSignatureNet(51,51)
hksNet.to(device)

HeatKernelSignatureNet(
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv1): Conv2d(1, 3, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1))
  (MLP): MLP(
    (model): Sequential(
      (fc1): Linear(in_features=48, out_features=100, bias=True)
      (tanh1): Tanh()
      (fc2): Linear(in_features=100, out_features=100, bias=True)
      (tanh2): Tanh()
      (fc3): Linear(in_features=100, out_features=100, bias=True)
      (tanh3): Tanh()
      (fc4): Linear(in_features=100, out_features=100, bias=True)
      (tanh4): Tanh()
      (fc5): Linear(in_features=100, out_features=100, bias=True)
      (tanh5): Tanh()
      (fc6): Linear(in_features=100, out_features=100, bias=True)
      (tanh6): Tanh()
      (fc7): Linear(in_features=100, out_features=100, bias=True)
      (tanh7): Tanh()
      (fc8): Linear(in_features=100, out_features=100, bias=

In [11]:
canvass_train_torch = torch.tensor(canvass_train).to(device).float()
canvass_train_torch = torch.reshape(canvass_train_torch,[num_drawings,1,canvas_width,canvas_height])

canvass_test_torch = torch.tensor(canvass_test).to(device).float()
canvass_test_torch = torch.reshape(canvass_test_torch,[num_drawings,1,canvas_width,canvas_height])

hkss_train_torch = torch.tensor(hkss_train).to(device).float()
hkss_test_torch = torch.tensor(hkss_test).to(device).float()

In [12]:
learning_rates = [0.005] * 2000 + [0.0005] * 2000 + [0.00005] * 2000

In [13]:
for i in range(len(learning_rates)):
    opt = torch.optim.AdamW(list(hksNet.parameters()),learning_rates[i])
    opt.zero_grad()

    pred_hks = hksNet(canvass_train_torch)
    loss = torch.mean(( (pred_hks - hkss_train_torch) / hkss_train_torch )**2)
    loss.backward()
    opt.step()

    if i % 10 == 0:
        print(i,loss.item())

0 1.056335210800171
10 0.10122397541999817
20 0.08471478521823883
30 0.08612339198589325
40 0.08495364338159561
50 0.08375366777181625
60 0.08886252343654633
70 0.10706138610839844
80 0.09943872690200806
90 0.11050180345773697
100 0.11535254120826721
110 0.11424514651298523
120 0.11260858923196793
130 0.10889837890863419
140 0.12171716243028641
150 0.3167561888694763
160 0.002728891558945179
170 0.028287742286920547
180 0.00711026880890131
190 0.00893318559974432
200 0.0036291901487857103
210 0.013589519076049328
220 0.035368066281080246
230 0.009859686717391014
240 0.018988071009516716
250 0.023300644010305405
260 0.003372594015672803
270 0.031923871487379074
280 0.025045059621334076
290 0.010029646568000317
300 0.00899331085383892
310 0.004351169802248478
320 0.008929792791604996
330 0.012313631363213062
340 0.0035912750754505396
350 0.00638597784563899
360 0.018342548981308937
370 0.02889324352145195
380 0.02755209058523178
390 0.045765701681375504
400 0.04997695982456207
410 0.0456

# Test

In [16]:
pred_hks_test = hksNet(canvass_test_torch)
loss_test = torch.mean(( ( pred_hks_test - hkss_test_torch) / hkss_test_torch )**2)
print(loss_test)

tensor(0.0001, device='cuda:0', grad_fn=<MeanBackward0>)


# Randomly sample a few to see what it looks like

In [22]:
print(pred_hks_test[15])
print(hkss_test_torch[15])

tensor([3.8615, 4.9981, 1.8424, 2.1658], device='cuda:0',
       grad_fn=<SelectBackward0>)
tensor([3.7095, 4.7786, 1.7998, 2.1027], device='cuda:0')


In [23]:
print(pred_hks_test[35])
print(hkss_test_torch[35])

tensor([3.9147, 5.0740, 1.8537, 2.1785], device='cuda:0',
       grad_fn=<SelectBackward0>)
tensor([3.9013, 5.0584, 1.8487, 2.1724], device='cuda:0')


In [24]:
print(pred_hks_test[60])
print(hkss_test_torch[60])

tensor([3.7077, 4.7664, 1.8068, 2.1088], device='cuda:0',
       grad_fn=<SelectBackward0>)
tensor([3.6584, 4.7068, 1.7852, 2.0823], device='cuda:0')


In [25]:
print(pred_hks_test[96])
print(hkss_test_torch[96])

tensor([3.7413, 4.8116, 1.8132, 2.1220], device='cuda:0',
       grad_fn=<SelectBackward0>)
tensor([3.7638, 4.8598, 1.8125, 2.1211], device='cuda:0')
